In [1]:
clean_up = True
%run StdPackages.ipynb
d['data'] = os.path.join(d['data'], 'IO2018')

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb2.gdx is still active and was not deleted.


# SmallGR: Data

The ```SmallGR``` model reduces the number of sectors from the ```GR18``` model to four sectors: Waste, energy, and two "other" sectors.

### 1. Load data

In [2]:
name = 'smallGR'
error = 1e-7 # tolerance when testing IO balance
db = GpyDB(pickle_path = os.path.join(d['data'], f'IO_GR18')) # GR18 IO data
db.name = f"IO_{name}"
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(2018,2051)}) # global settings used throughout; interest rates, long run growth rates, time index etc.

### 2. Aggregation

### 2.1 Sector aggregation

Aggregate sectors further: Waste, energy, two "other" sectors.

In [7]:
w = pd.MultiIndex.from_tuples([(k, 'waste') for k in ('38391','38392','38394','38395')], names = ['s','ss'])
e = pd.MultiIndex.from_tuples([(k, 'energy') for k in ('19000','35011','350020')], names = ['s','ss'])
non_we = adj.rc_pd(db.get('s_p'), ('not',('or',[w.levels[0], e.levels[0]])))
o = pd.MultiIndex.from_tuples([(k, 'other1') for k in non_we[0:round(len(non_we)/2)]]+
                              [(k, 'other2') for k in non_we[round(len(non_we)/2):]], names = ['s','ss'])
m = pd.MultiIndex.from_tuples(np.hstack([x.values for x in (w,e,o)]), names = ['s','ss']) # full mapping

Aggregate durable sectors into one:

In [8]:
mDur = pd.MultiIndex.from_product([db.get('s_i'), ['I_iK']], names = ['s','ss'])

For sectors not included here do nothing: This is done by including (x,x) syntax.

In [9]:
m_s = m.union(mDur)
m_sector = m_s.union(pd.MultiIndex.from_arrays([adj.rc_pd(db.get('s'), ('not', m_s.levels[0])), adj.rc_pd(db.get('s'), ('not', m_s.levels[0])).rename('ss')]))
aggregateDB.aggDB(db, m_sector, aggBy='s',replaceWith='ss')

We do the same to the type of goods:

In [10]:
mDur_n = mDur.set_levels([l.str.strip('I_') for l in mDur.levels]).rename(['n','ntemp'])
m_goods = m.rename(['n','ntemp']).union(m.set_levels([level+'_F' for level in m.levels]).rename(['n','ntemp'])).union(mDur_n).union(mDur.rename(['n','ntemp']))
m_goods = m_goods.union(pd.MultiIndex.from_arrays([adj.rc_pd(db.get('n'), ('not', m_goods.levels[0])),
                                                   adj.rc_pd(db.get('n'), ('not', m_goods.levels[0])).rename('ntemp')]))
aggregateDB.aggDB(db, m_goods)

### 3. Redefine sparse dummies

The dummies ```dImport,dImport_dom,dImport_for``` should indicate whether or not a specific sector draws on foreign+domestic goods, only domestic, or only foreign goods (for each goods category). The simple aggregation procedure does not keep these sparsity patterns. So, here we redo them:

Define the subset ```dImport``` as  the sector, domestic good, foreign good (s,n,nn) combinations in data - i.e. where the sector demands both the domestic and foreign product:

In [11]:
vD_dom = adjMultiIndex.applyMult(adj.rc_pd(db.get('vD'), db.get('n_p')), db.get('dom2for')) # demand for domestic goods mapped to foreign goods types
vD_for = adj.rc_pd(db.get('vD'), db.get('n_F')).rename_axis(index={'n':'nn'}) # demand for foreign goods
db['dImport'] = adj.rc_pd(vD_dom, vD_for).reorder_levels(['s','n','nn']).index

Define the subset ```dImport_dom``` as the sector, domestic good combination (s,n) where the sector only demands the domestic and not the corresponding foreign good:

In [12]:
db['dImport_dom'] = adj.rc_pd(vD_dom, ('not', vD_for)).droplevel('nn').reorder_levels(['s','n']).index

Define the subset ```dImport_for``` as the sector, foreign good combinations (s,n) where the sector only demand the foreign and not the corresponding domestic good:

In [13]:
db['dImport_for'] = adj.rc_pd(vD_for, ('not', db['dImport'])).index.rename(['s','n']).reorder_levels(['s','n'])

### 4. Redefining prices and rates:

The symbols in the database are per default summed up when aggregating. This makes sense when we are talking about quantitiy/value symbols, but not when we are using rates (incl. prices). Instead of finding suitable aggregation weights for each symbol, here we simply reset them to standard levels. **NB: This means that any non-standard price definitions are overwritten here**.

In [14]:
db['rDepr'] = db.get('vD_depr') / db.get('vD_dur')
db['p'] = pd.Series(1, index = db.get('vS').index.levels[-1].union(db.get('n_F')))
db['pD_dur'] = adjMultiIndex.applyMult(db.get('p').rename_axis(index = {'n':'nn'}), db.get('dur2inv')).dropna().droplevel('nn') * (glob.db['R_LR'].vals/(1+glob.db['infl_LR'].vals)+db.get('rDepr')-1)
db['pD'] = adjMultiIndex.bc(db.get('p'), adj.rc_pd(db.get('qD'), ('not', db.get('dur_p')))).reorder_levels(db['qD'].domains).rename('pD') # span the pure prices 'p' to fit entire qD domain
db['pS'] = adjMultiIndex.bc(adj.rc_pd(db.get('p'), ('not', db.get('n_F'))), db.get('qS')).reorder_levels(db['qS'].domains).rename('qS') # span 'p' to fit domain of qS. Drop prices on foreign goods.

### Export

In [15]:
aggregateDB.readSets(db) # read sets from the symbols in data
db.export(repo = d['data'])
with open(f"{d['data']}\\glob_{name}", "wb") as file:
    pickle.dump(glob,file)